In [1]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from _version import __version__
from data import *
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

In [3]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)

In [6]:
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()


In [7]:
# get the completed experiments with code version 5
df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed
0,4e5575c5-b509-43bf-9eb3-92c2d401fa13,test,100.0,6,2023-03-25 11:20:46.806311,5,5,"[0.1, 0.2, 0.4]","[0.1, 0.2, 0.4]","[0.1, 0.2, 0.4]",0.1,10,"[""test"", ""test2""]",1
1,01508815-e267-47db-ab0f-dd43a3203817,test,100.0,6,2023-03-25 11:23:13.936436,5,5,"[0.1, 0.2, 0.4]","[0.1, 0.2, 0.4]","[0.1, 0.2, 0.4]",0.1,10,"[""test"", ""test2""]",1


In [10]:
df_state_evolution

,id,code_version,duration,experiment_id,generalization_error,date,sigma,q,m,initial_condition,rho_w_star,alpha,epsilon,tau,lam,abs_tol,min_iter,max_iter,blend_fpe,int_lims
0,96750187-ee31-4b30-b37f-f134f73d1b21,6,100.0,4e5575c5-b509-43bf-9eb3-92c2d401fa13,0.428217,2023-03-25 11:20:46.806889,0.1,0.1,0.1,"[0.1, 0.1, 0.1]",1.0,0.1,0.1,0.1,0.1,0.1,0,0,0.0,0.0
1,c06d6e5a-a5dd-428d-a85c-5046c5016ef4,6,100.0,01508815-e267-47db-ab0f-dd43a3203817,0.428217,2023-03-25 11:23:13.940200,0.1,0.1,0.1,"[0.1, 0.1, 0.1]",1.0,0.1,0.1,0.1,0.1,0.1,0,0,0.0,0.0


In [8]:
df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]

,id,code_version,duration,experiment_id,generalization_error,date,sigma,q,m,initial_condition,rho_w_star,alpha,epsilon,tau,lam,abs_tol,min_iter,max_iter,blend_fpe,int_lims


In [9]:
df_erm

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
0,3b594235-c6d0-49a8-9d58-d0dbdee3f975,100.0,6,4e5575c5-b509-43bf-9eb3-92c2d401fa13,0.978205,1.128952,-0.536413,-0.510442,0.1,0.1,0.29,0.670518,2023-03-25 11:20:46.807547,test,0.53,10,0.1,10.0,0.736177
1,aa493e0c-be07-485d-9e6a-ce4ef8340d69,100.0,6,01508815-e267-47db-ab0f-dd43a3203817,1.586212,0.687552,0.261899,0.250785,0.1,0.1,0.35,0.419311,2023-03-25 11:23:13.941090,test,0.57,10,0.1,10.0,0.795312


In [36]:
# rename experimend_id to experiment_id
df_erm = df_erm.rename(columns={"experimend_id":"experiment_id"})

In [ ]:
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]

In [48]:
# merge df_erm and df_state_evolution on experiment id efd6ee50-96f6-4ddd-9a84-ce6c74469637
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
# print the number of rows in each dataframe
print("erm: %d, state_evolution: %d" % (erm.shape[0],state_evolution.shape[0]))
# round alpha and epsilon to 2 decimals
erm["alpha"] = erm["alpha"].round(2)
erm["epsilon"] = erm["epsilon"].round(2)
state_evolution["alpha"] = state_evolution["alpha"].round(2)
state_evolution["epsilon"] = state_evolution["epsilon"].round(2)
# keep alpha, epsilon and the generalization errors
erm = erm[["alpha","epsilon","generalization_error_erm","generalization_error_overlap"]]
state_evolution = state_evolution[["alpha","epsilon","generalization_error"]]

# join the dataframes on alpha an epsilon and add suffixes to the columns
df = erm.merge(state_evolution, on=["alpha","epsilon"], suffixes=("_erm","_state_evolution"))
# print the number of rows in the joined dataframe
print("joined: %d" % df.shape[0])
df

erm: 6, state_evolution: 6
joined: 6


/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["alpha"] = state_evolution["alpha"].round(2)
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["epsilon"] = state_evolution["epsilon"].round(2)


,alpha,epsilon,generalization_error_erm,generalization_error_overlap,generalization_error
0,0.64,0.02,0.46955,0.440456,0.122996
1,0.64,0.00,0.46935,0.416841,0.123619
2,2.28,0.02,0.44400,0.351962,0.066441
3,5.00,0.00,0.41380,0.300245,0.045324
4,5.00,0.02,0.41235,0.287456,0.045062
5,2.28,0.00,0.44595,0.386968,0.066818
